In [95]:
import requests
import pandas as pd
from GetWbData import GetWbData
from GetWbIndicators import GetWbIndicators

### Question 1
Use the API to get all available data for the GDP per capita, PPP (constant 2017 international $) indicator. Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [4]:
# variables
lst = []
endpoint = 'http://api.worldbank.org/v2/country/all/indicator/'
indicator = 'NY.GDP.PCAP.PP.KD'
params = {
    'format': 'json',
    'per_page': 50
}
url = endpoint + indicator

# get number of pages
res = requests.get(url, params).json()
pages = res[0]['pages']

for page in range(1, pages + 1):
    params['page'] = page
    res = requests.get(url, params).json()
    lst.append(res[1])

gdp = pd.json_normalize([r for d in lst for r in d])

### Question 2
Now, use the API to get all available data for Life expectancy at birth, total (years). This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.

In [8]:
# variables
lst = []
endpoint = 'http://api.worldbank.org/v2/country/all/indicator/'
indicator = 'SP.DYN.LE00.IN'
params = {
    'format': 'json',
    'per_page': 50
}
url = endpoint + indicator

# get number of pages
res = requests.get(url, params).json()
pages = res[0]['pages']

for page in range(1, pages + 1):
    params['page'] = page
    res = requests.get(url, params).json()
    lst.append(res[1])

le = pd.json_normalize([r for d in lst for r in d])

### Question 3
Merge the two results DataFrames together. You may want to rename or drop columns prior to merging.

In [19]:
# drop empty columns
gdp = gdp.drop(['unit', 'obs_status', 'decimal', 'indicator.id', 'indicator.value'], axis = 1)

In [22]:
# rename remaining columns
gdp = gdp.rename(columns = {'value': 'gdp_per_capita', 'country.id': 'country_id', 'country.value': 'country_name'})

In [24]:
gdp.head(1)

,countryiso3code,date,gdp_per_capita,country_id,country_name
0,AFE,2022,3553.91337,ZH,Africa Eastern and Southern


In [27]:
le = le.drop(['unit', 'obs_status', 'decimal', 'indicator.id', 'indicator.value'], axis = 1)

In [29]:
le = le.rename(columns = {'value': 'life_expectancy', 'country.id': 'country_id', 'country.value': 'country_name'})

In [30]:
le.head(1)

,countryiso3code,date,life_expectancy,country_id,country_name
0,AFE,2022,NaN,ZH,Africa Eastern and Southern


In [31]:
gdp_le = pd.merge(gdp, le, on = ['countryiso3code', 'date', 'country_id', 'country_name'], how = 'inner')

In [32]:
gdp_le.head(1)

,countryiso3code,date,gdp_per_capita,country_id,country_name,life_expectancy
0,AFE,2022,3553.91337,ZH,Africa Eastern and Southern,NaN


### Question 4
You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the Country API. Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

In [89]:
# variables
lst = []
endpoint = 'http://api.worldbank.org/v2/country/all/'
# indicator = 'SP.DYN.LE00.IN'
params = {
    'format': 'json',
    'per_page': 50
}
url = endpoint # + indicator

# get number of pages
res = requests.get(url, params).json()
pages = res[0]['pages']

for page in range(1, pages + 1):
    params['page'] = page
    res = requests.get(url, params).json()
    lst.append(res[1])

country = pd.json_normalize([r for d in lst for r in d])

In [90]:
country.head(1)

,id,iso2Code,name,capitalCity,longitude,latitude,region.id,region.iso2code,region.value,adminregion.id,adminregion.iso2code,adminregion.value,incomeLevel.id,incomeLevel.iso2code,incomeLevel.value,lendingType.id,lendingType.iso2code,lendingType.value
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,LCN,ZJ,Latin America & Caribbean,,,,HIC,XD,High income,LNX,XX,Not classified


In [91]:
country = country.rename({'id': 'countryiso3code', 
                          'iso2Code': 'country_id', 
                          'name': 'country_name',
                          'capitalCity': 'capital_city',
                          'region.id': 'region_id', 
                          'region.iso2code': 'region_code', 
                          'region.value': 'region_name', 
                          'adminregion.id': 'admin_region_id', 
                          'adminregion.iso2code': 'admin_region_code',
                          'adminregion.value': 'admin_region_name',
                          'incomeLevel.id': 'income_level_id',
                          'incomeLevel.iso2code': 'income_level_code',
                          'incomeLevel.value': 'income_level',
                          'lendingType.id': 'lending_type_id',
                          'lendingType.iso2code': 'lending_type_code',
                          'lendingType.value': 'lending_type'}, axis = 1)

In [92]:
# drop rows with null (empty string) longitude to eliminate non-countries
# country = country[country['longitude']!= '']

# getting about 100 less than Michael, should drop 'region_id == "Aggregate" instead'
country = country[country['region_id']!= 'Aggregate']

In [93]:
gdp_le_country = pd.merge(gdp_le, country, on = ['countryiso3code', 'country_name', 'country_id'], how = 'inner')

In [94]:
gdp_le_country.shape

(16317, 21)

### Before tackling the bonus questions I turned the loop I was using into a function and put it in a module.

### Bonus Question 1
Adjust your request so that it returns data just for the United States.

In [66]:
# trying this out with life expectancy, it will work the same for gdp
params = {
    'format': 'json',
    'per_page': 50
}

le_us = GetWbData(params, country = 'US', indicator = 'SP.DYN.LE00.IN')

le_us.head()

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,USA,2022,NaN,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",US,United States
1,USA,2021,76.329268,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",US,United States
2,USA,2020,76.980488,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",US,United States
3,USA,2019,78.787805,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",US,United States
4,USA,2018,78.639024,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",US,United States


### Bonus Question 2
Adjust your request so that it returns data just for the United States for the year 2021.

In [96]:
# trying this out with life expectancy, it will work the same for gdp
params = {
    'format': 'json',
    'per_page': 50,
    'date': 2021
}

le_us_2021 = GetWbData(params, country = 'US', indicator = 'SP.DYN.LE00.IN')

le_us_2021.head()

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,USA,2021,76.329268,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",US,United States


### Bonus Question 3
Adjust your request so that it returns data just for the United States for the years 2000 through 2021.

In [68]:
params = {
    'format': 'json',
    'per_page': 50,
    'date': '2000:2021'
}

le_us_2000_2021 = GetWbData(params, country = 'US', indicator = 'SP.DYN.LE00.IN')

le_us_2000_2021.head()

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,USA,2021,76.329268,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",US,United States
1,USA,2020,76.980488,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",US,United States
2,USA,2019,78.787805,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",US,United States
3,USA,2018,78.639024,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",US,United States
4,USA,2017,78.539024,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",US,United States


### Bonus Question 4
Adjust your request so that it returns data for the United States and Canada for the years 2000 through 2021.

In [70]:
params = {
    'format': 'json',
    'per_page': 50,
    'date': '2000:2021'
}

le_us_ca_2000_2021 = GetWbData(params, country = 'US;CA', indicator = 'SP.DYN.LE00.IN')

le_us_ca_2000_2021.head()

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,CAN,2021,82.596610,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",CA,Canada
1,CAN,2020,81.670488,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",CA,Canada
2,CAN,2019,82.228780,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",CA,Canada
3,CAN,2018,81.843902,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",CA,Canada
4,CAN,2017,81.832927,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",CA,Canada


### Bonus question 5
If you haven't already done so and you would like to get some additional practice using loops, use the page parameter in order to pull all records. Do not change the value of the per_page parameter. You will likely need to utilize a loop of some kind in order to pull all records.

Went ahead and did this at the beginning.

### Bonus Question 6
The endpoint "http://api.worldbank.org/v2/indicator" will return information about all available indicators. See Indicator API Queries. Use this endpoint to find the code for the "Public Expenditure on Education (% GDP)" indicator.

Need to adjust my function for this one! Go back and refactor so everything can be in a single function, I'm just up against the clock at the moment.

In [87]:
params = {
    'format': 'json',
    'per_page': 50
}

indicators = GetWbIndicators(params)

indicators.head()

,id,name,unit,sourceNote,sourceOrganization,topics,source.id,source.value
0,1.0.HCount.1.90usd,Poverty Headcount ($1.90 a day),,The poverty headcount index measures the propo...,LAC Equity Lab tabulations of SEDLAC (CEDLAS a...,"[{'id': '11', 'value': 'Poverty '}]",37,LAC Equity Lab
1,1.0.HCount.2.5usd,Poverty Headcount ($2.50 a day),,The poverty headcount index measures the propo...,LAC Equity Lab tabulations of SEDLAC (CEDLAS a...,"[{'id': '11', 'value': 'Poverty '}]",37,LAC Equity Lab
2,1.0.HCount.Mid10to50,Middle Class ($10-50 a day) Headcount,,The poverty headcount index measures the propo...,LAC Equity Lab tabulations of SEDLAC (CEDLAS a...,"[{'id': '11', 'value': 'Poverty '}]",37,LAC Equity Lab
3,1.0.HCount.Ofcl,Official Moderate Poverty Rate-National,,The poverty headcount index measures the propo...,LAC Equity Lab tabulations of data from Nation...,"[{'id': '11', 'value': 'Poverty '}]",37,LAC Equity Lab
4,1.0.HCount.Poor4uds,Poverty Headcount ($4 a day),,The poverty headcount index measures the propo...,LAC Equity Lab tabulations of SEDLAC (CEDLAS a...,"[{'id': '11', 'value': 'Poverty '}]",37,LAC Equity Lab
